# Data Preparation

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
!pip install transformers
import torch
import transformers
from transformers import AutoModel,AutoTokenizer

     |████████████████████████████████| 778kB 4.4MB/s 
     |████████████████████████████████| 1.1MB 11.7MB/s 
     |████████████████████████████████| 890kB 24.6MB/s 
     |████████████████████████████████| 3.0MB 31.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=c92374b11a0e54e11443eba4d3974da21f5d101b80dfe2bf68d88eef5207cde5
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


## Getting emotions labels for prediction from the Arabic dataset
### 8 emotions in total

In [ ]:
import pandas as pd

path = "/content/drive/My Drive/Colab Notebooks/NLP/Project/arabic_emotions_dataset.csv"
df = pd.read_csv(path, index_col=0, engine="python")
df.columns = [c.strip() for c in df.columns]
df = df[(df["TWEET"].notnull()) & (df["LABEL"].notnull())]

In [ ]:
set(df["LABEL"])

{'anger', 'fear', 'joy', 'love', 'none', 'sadness', 'surprise', 'sympathy'}

#Data Processing
#EmoBank
###Converting VAD to emotions
According to the paper: James A. Russell and Albert Mehrabian. 1977. Evidence for a three-factor theory of emotions. Journal
of Research in Personality, 11(3):273–294.

In [ ]:
import pandas as pd

eb_path = "/content/drive/My Drive/Colab Notebooks/NLP/Project/emobank.csv"
eb = pd.read_csv(eb_path, index_col=0, engine="python")

In [ ]:
emotions = set(df["LABEL"])

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Normalizing VAD values
scaler = MinMaxScaler((-1, 1))
scaled = scaler.fit_transform(eb[["V", "A", "D"]])
cols = ["NORM_V", "NORM_A", "NORM_D"]
for i, c in enumerate(cols):
  eb[c] = [s[i] for s in scaled]

In [ ]:
eb.head()

,split,V,A,D,text,NORM_V,NORM_A,NORM_D
id,,,,,,,,
110CYL068_1036_1079,train,3.00,3.00,3.20,"Remember what she said in my last letter? """,0.058824,-0.076923,0.173554
110CYL068_1079_1110,test,2.80,3.10,2.80,If I wasn't working here.,-0.058824,0.000000,-0.157025
110CYL068_1127_1130,train,3.00,3.00,3.00,"..""",0.058824,-0.076923,0.008264
110CYL068_1137_1188,train,3.44,3.00,3.22,Goodwill helps people get off of public assist...,0.317647,-0.076923,0.190083
110CYL068_1189_1328,train,3.55,3.27,3.46,Sherry learned through our Future Works class ...,0.382353,0.130769,0.388430


In [ ]:
import numpy as np
from sklearn.metrics.pairwise import euclidean_distances
from scipy.spatial import distance

emo2vad = {
    "anger": (-0.51, 0.59, 0.25), #82
    "fear": (-0.64, 0.60, -0.43), #101
    "joy": (0.76, 0.48, 0.35), #20
    "love": (0.87, 0.54, -0.18), #47
    "none": (0, 0, 0),
    "sadness": (-0.63, -0.27, -0.33), #151
    "surprise": (0.40, 0.67, -0.13), #52
    "sympathy": (0.64, 0.35, 0.24), #34
}
def convertVadToEmotion(vad):
  """
  Convert normalized VAD to emotion label.

  Parameters
  ----------
  vad : tuple
    VAD values.
  
  Returns
  -------
  str
    Closest emotion label.
  """
  closest_emotion = None
  closest_dist = None
  for emo, emo_vad in emo2vad.items():
    dist = distance.euclidean(np.array(vad), np.array(emo_vad))
    if closest_dist is None or closest_dist > dist:
      closest_dist = dist
      closest_emotion = emo

  return closest_emotion

In [ ]:
eb["emotion"] = eb.apply(lambda row: convertVadToEmotion((row["NORM_V"], row["NORM_A"], row["NORM_D"])), axis=1)
print(eb["emotion"])

id
110CYL068_1036_1079        none
110CYL068_1079_1110        none
110CYL068_1127_1130        none
110CYL068_1137_1188        none
110CYL068_1189_1328    sympathy
                         ...   
wwf12_4531_4624            none
wwf12_501_591          sympathy
wwf12_592_691              none
wwf12_702_921          sympathy
wwf12_932_1053         sympathy
Name: emotion, Length: 10062, dtype: object


In [ ]:
eb["emotion"].value_counts()

none        8826
sympathy     573
sadness      247
anger        204
fear          99
surprise      58
joy           49
love           6
Name: emotion, dtype: int64

###Translating EmoBank to arabic

In [ ]:
!pip install googletrans

     |████████████████████████████████| 61kB 3.1MB/s 
     |████████████████████████████████| 51kB 6.2MB/s 
     |████████████████████████████████| 952kB 22.2MB/s 
     |████████████████████████████████| 71kB 7.7MB/s 
     |████████████████████████████████| 61kB 6.8MB/s 
     |████████████████████████████████| 102kB 9.5MB/s 
  Created wheel for googletrans: filename=googletrans-3.0.0-cp36-none-any.whl size=15736 sha256=b5253e57d38455198735b872c102cf89fb250ad4a811de88072f2139187631cc
  Stored in directory: /root/.cache/pip/wheels/28/1a/a7/eaf4d7a3417a0c65796c547cff4deb6d79c7d14c2abd29273e
  Created wheel for contextvars: filename=contextvars-2.4-cp36-none-any.whl size=7666 sha256=497927de52d667853ea71ecbf999202372d9218e32aaea86a3da2490b1fd4752
  Stored in directory: /root/.cache/pip/wheels/a5/7d/68/1ebae2668bda2228686e3c1cf16f2c2384cea6e9334ad5f6de
Successfully built googletrans contextvars


In [ ]:
from googletrans import Translator

In [ ]:
translator = Translator()
eb["translate"] = eb["text"].apply(lambda t: translator.translate(t, src="en", dest="ar"))

In [ ]:
eb.head()

,split,V,A,D,text,translate
id,,,,,,
110CYL068_1036_1079,train,3.00,3.00,3.20,"Remember what she said in my last letter? ""","Translated(src=en, dest=ar, text=أتذكر ما قالت..."
110CYL068_1079_1110,test,2.80,3.10,2.80,If I wasn't working here.,"Translated(src=en, dest=ar, text=إذا لم أكن أع..."
110CYL068_1127_1130,train,3.00,3.00,3.00,"..""","Translated(src=en, dest=ar, text=.. "", pronunc..."
110CYL068_1137_1188,train,3.44,3.00,3.22,Goodwill helps people get off of public assist...,"Translated(src=en, dest=ar, text=حسن النية يسا..."
110CYL068_1189_1328,train,3.55,3.27,3.46,Sherry learned through our Future Works class ...,"Translated(src=en, dest=ar, text=علمت شيري من ..."


In [ ]:
eb["translated_text"] = eb["translate"].apply(lambda t: t.text)

In [ ]:
eb.head()

,split,V,A,D,text,translate,translated_text
id,,,,,,,
110CYL068_1036_1079,train,3.00,3.00,3.20,"Remember what she said in my last letter? ""","Translated(src=en, dest=ar, text=أتذكر ما قالت...","أتذكر ما قالت في رسالتي الأخيرة؟ """
110CYL068_1079_1110,test,2.80,3.10,2.80,If I wasn't working here.,"Translated(src=en, dest=ar, text=إذا لم أكن أع...",إذا لم أكن أعمل هنا.
110CYL068_1127_1130,train,3.00,3.00,3.00,"..""","Translated(src=en, dest=ar, text=.. "", pronunc...",".. """
110CYL068_1137_1188,train,3.44,3.00,3.22,Goodwill helps people get off of public assist...,"Translated(src=en, dest=ar, text=حسن النية يسا...",حسن النية يساعد الناس على الحصول على المساعدة ...
110CYL068_1189_1328,train,3.55,3.27,3.46,Sherry learned through our Future Works class ...,"Translated(src=en, dest=ar, text=علمت شيري من ...",علمت شيري من خلال صف أعمال المستقبل أنها تستطي...


###Adding column for recognizing whether all sentence in Arabic

In [ ]:
import unicodedata
from string import punctuation

def is_all_in_arabic(s):
  """
  Detecting whether a string is completely in English.

  Parameters
  ----------
  s : str
    String.

  Returns
  -------
  bool
    Whether the input is completely in English.
  """
  for c in s:
    if c in punctuation or c == " ":
      continue
    name = unicodedata.name(c).lower()
    if 'arabic' not in name and 'digit' not in name:
      return False
  return True

eb["is_all_in_arabic"] = eb["translated_text"].apply(is_all_in_arabic)

In [ ]:
eb[eb["is_all_in_arabic"] == True]

,split,V,A,D,text,translate,translated_text,is_all_in_arabic
id,,,,,,,,
110CYL068_1036_1079,train,3.00,3.00,3.20,"Remember what she said in my last letter? ""","Translated(src=en, dest=ar, text=أتذكر ما قالت...","أتذكر ما قالت في رسالتي الأخيرة؟ """,True
110CYL068_1079_1110,test,2.80,3.10,2.80,If I wasn't working here.,"Translated(src=en, dest=ar, text=إذا لم أكن أع...",إذا لم أكن أعمل هنا.,True
110CYL068_1127_1130,train,3.00,3.00,3.00,"..""","Translated(src=en, dest=ar, text=.. "", pronunc...",".. """,True
110CYL068_1137_1188,train,3.44,3.00,3.22,Goodwill helps people get off of public assist...,"Translated(src=en, dest=ar, text=حسن النية يسا...",حسن النية يساعد الناس على الحصول على المساعدة ...,True
110CYL068_1189_1328,train,3.55,3.27,3.46,Sherry learned through our Future Works class ...,"Translated(src=en, dest=ar, text=علمت شيري من ...",علمت شيري من خلال صف أعمال المستقبل أنها تستطي...,True
...,...,...,...,...,...,...,...,...
wwf12_4173_4229,train,3.12,2.75,3.00,"Sincerely, Kathryn S. Fuller President","Translated(src=en, dest=ar, text=مع خالص التقد...",مع خالص التقدير ، كاثرين س. فولر رئيسة,True
wwf12_4531_4624,train,3.00,3.50,3.00,Please let it be a constant reminder of all yo...,"Translated(src=en, dest=ar, text=يرجى أن يكون ...",يرجى أن يكون تذكيرًا دائمًا بكل ما تفعله للحفا...,True
wwf12_501_591,train,3.80,3.40,3.60,That�s why I want to extend my appreciation fo...,"Translated(src=en, dest=ar, text=لهذا السبب أو...",لهذا السبب أود أن أعرب عن تقديري لمساهمتكم الأ...,True


## Storing in dataframe

In [ ]:
eb.to_csv("/content/drive/My Drive/Colab Notebooks/NLP/Project/emobank_translated.csv")

In [ ]:
eb_path = "/content/drive/My Drive/Colab Notebooks/NLP/Project/emobank_translated.csv"
eb = pd.read_csv(eb_path, index_col=0, engine="python")

In [ ]:
eb.head()

,split,V,A,D,text,translate,translated_text,is_all_in_arabic,NORM_V,NORM_A,NORM_D,emotion
id,,,,,,,,,,,,
110CYL068_1036_1079,train,3.00,3.00,3.20,"Remember what she said in my last letter? ""","Translated(src=en, dest=ar, text=أتذكر ما قالت...","أتذكر ما قالت في رسالتي الأخيرة؟ """,True,0.058824,-0.076923,0.173554,none
110CYL068_1079_1110,test,2.80,3.10,2.80,If I wasn't working here.,"Translated(src=en, dest=ar, text=إذا لم أكن أع...",إذا لم أكن أعمل هنا.,True,-0.058824,0.000000,-0.157025,none
110CYL068_1127_1130,train,3.00,3.00,3.00,"..""","Translated(src=en, dest=ar, text=.. "", pronunc...",".. """,True,0.058824,-0.076923,0.008264,none
110CYL068_1137_1188,train,3.44,3.00,3.22,Goodwill helps people get off of public assist...,"Translated(src=en, dest=ar, text=حسن النية يسا...",حسن النية يساعد الناس على الحصول على المساعدة ...,True,0.317647,-0.076923,0.190083,none
110CYL068_1189_1328,train,3.55,3.27,3.46,Sherry learned through our Future Works class ...,"Translated(src=en, dest=ar, text=علمت شيري من ...",علمت شيري من خلال صف أعمال المستقبل أنها تستطي...,True,0.382353,0.130769,0.388430,sympathy


In [ ]:
len(eb[eb["is_all_in_arabic"] == True])

4040

#Data loading and splitting

In [ ]:
LABELS = ['sadness', 'none', 'sympathy', 'fear', 'surprise', 'joy', 'love', 'anger']
NUM_LABELS = len(LABELS)

In [ ]:
import logging
logging.basicConfig(level=logging.ERROR)

In [ ]:
import pandas as pd

def load_arabic_dataset():
  """
  Load the Arabic dataset.

  Returns
  -------
  tuple, (np.ndarray, np.ndarray)
    Sentences and labels.
  """
  path = "/content/drive/My Drive/Colab Notebooks/NLP/Project/arabic_emotions_dataset.csv"
  df = pd.read_csv(path, index_col=0, engine="python")
  df.columns = [c.strip() for c in df.columns]
  df = df[(df["TWEET"].notnull()) & (df["LABEL"].notnull())]
  sentences = df["TWEET"].values
  labels = df["LABEL"].values

  return sentences, labels


def load_eb_arabic_dataset():
  """
  Load EmoBank dataset in Arabic.

  Returns
  -------
  tuple, (np.ndarray, np.ndarray)
    Sentences and labels where labels here are normalized VAD values.
  """
  path = "/content/drive/My Drive/Colab Notebooks/NLP/Project/emobank_translated.csv"
  df = pd.read_csv(path, index_col=0, engine="python")
  mask = (df["is_all_in_arabic"] == True)
  sentences = df.loc[mask, "translated_text"].values
  labels = df.loc[mask, ["NORM_V", "NORM_A", "NORM_D"]].values

  return sentences, labels


def load_eb_english_emotion_dataset():
  """
  Load EmoBank dataset in English.

  Returns
  -------
  tuple, (np.ndarray, np.ndarray)
    Sentences and labels.
  """
  path = "/content/drive/My Drive/Colab Notebooks/NLP/Project/emobank_translated.csv"
  df = pd.read_csv(path, index_col=0, engine="python")
  sentences = df["text"].values
  labels = df["emotion"].values

  return sentences, labels

In [ ]:
import torch.nn as nn

def tokenize(tokenizer, sentences, labels, is_emotions=True):
  """
  Tokenize all of the sentences and map the tokens to thier word IDs.

  Parameters
  ----------
  tokenizer : transformers.PreTrainedTokenizer
    Tokenizer.
  sentences : np.ndarray
    Sentences.
  labels : np.ndarray
    Labels.
  is_emotions : bool
    Whether the labels are in emotions labels or VAD.
    Optional; default: True.

  Returns
  -------
  tuple, (3 tensors)
    input_ids (word IDs for every sentence), attention_masks, labels
  """
  input_ids = []
  attention_masks = []

  # For every sentence
  for sent in sentences:
    encoded_dict = tokenizer.encode_plus(
      sent,                         # Sentence to encode
      add_special_tokens = True,    # Add '[CLS]' and '[SEP]'
      max_length = 64,              # Pad & truncate all sentences.
      pad_to_max_length = True,     # Pad up to max length of 64
      return_attention_mask = True, # Construct attn. masks.
      return_tensors = 'pt',        # Return pytorch tensors.
    )
    
    # Add the encoded sentence to the list   
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding)
    attention_masks.append(encoded_dict['attention_mask'])

  # Convert the lists into tensors
  input_ids = torch.cat(input_ids, dim=0)
  attention_masks = torch.cat(attention_masks, dim=0)

  # For emotions labels we need to convert to one-hot
  if is_emotions:
    labels = torch.tensor([LABELS.index(l) for l in labels])
    labels = nn.functional.one_hot(labels, num_classes=NUM_LABELS)

  # For VAD this is already float
  else:
    labels = torch.tensor(labels)

  # Print sentence 0, now as a list of IDs.
  print('Original: ', sentences[0])
  print('Token IDs:', input_ids[0])

  return input_ids, attention_masks, labels

In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, TensorDataset, random_split

def load_split_dataset(input_ids, attention_masks, labels, to_split=False):
  """
  Load (and potentially split) dataset.

  Parameters
  ----------
  input_ids : tensor
    Sentences words IDs.
  attention_masks : tensor
    Attenstion masks.
  labels : tensor
    Labels.
  to_split : bool
    Whether to also split dataset to train and validation.

  Returns
  -------
  tuple, (torch.utils.data.DataLoader, torch.utils.data.DataLoader or None)
    If split, returns train data loader and validatoin data loader.
    Else, returns the whole dataset in data loader.
  """
  # Combine the training inputs into a TensorDataset
  dataset = TensorDataset(input_ids, attention_masks, labels)

  if to_split:
    # Create a 90-10 train-validation split

    # Calculate the number of samples to include in each set
    train_size = int(0.9 * len(dataset))
    val_size = len(dataset) - train_size

    # Divide the dataset by randomly selecting samples
    train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

  else:
    # Take the whole dataset
    train_dataset = dataset

  # The DataLoader needs to know our batch size for training, so we specify it 
  # here. For fine-tuning BERT on a specific task, we saw that it is recommended
  # to use a batch size of 16 or 32
  batch_size = 32

  # Create the DataLoaders for our training and validation sets
  # We'll take training samples in random order
  train_dataloader = DataLoader(
              train_dataset,
              sampler = RandomSampler(train_dataset), # Select batches randomly
              batch_size = batch_size
          )

  if to_split:
    # For validation the order doesn't matter, so we'll just read them sequentially
    validation_dataloader = DataLoader(
                val_dataset,
                sampler = SequentialSampler(val_dataset), # Pull out batches sequentially
                batch_size = batch_size
            )
  else:
    validation_dataloader = None
    
  return train_dataloader, validation_dataloader

#Train function

In [ ]:
def train(net, criterion, opti, train_loader, val_loader, is_emotions=True):
  """
  Training procedure.

  Parameters
  ----------
  net : torch.nn.Module
    Network.
  criterion : torch.nn.MSELoss
    Criterion.
  opti : torch.optim
    Optimizer.
  train_loader : torch.utils.data.DataLoader
    Train data loader.
  val_loader : torch.utils.data.DataLoader
    Validation data loader.
  is_emotions : bool
    Whether the labels are in emotions labels or VAD.
    Optional; default: True.
  """
  for ep in range(5):
    for it, (seq, attn_masks, labels) in enumerate(train_loader):
      # Clear gradients
      opti.zero_grad()  
      # Converting these to cuda tensors
      seq, attn_masks, labels = seq.cuda(), attn_masks.cuda(), labels.cuda()

      # Obtaining the outputs from the model
      outputs = net(seq, attn_masks)

      # Computing loss
      # For emotion labels
      if is_emotions:
        loss = criterion(outputs.squeeze(-1).float(), labels.float())
      # For VAD values
      else:
        loss = criterion(outputs.squeeze(-1).double(), labels.double())

      # Backpropagating the gradients
      loss.backward()

      # Optimization step
      opti.step()

      if (it + 1) % 1 == 0:
          print("Iteration {} of epoch {} complete. Loss : {}".format(it+1, ep+1, loss.item()))

#Arabic dataset model

In [ ]:
sentences, labels = load_arabic_dataset()

In [ ]:
bert_name="asafaya/bert-base-arabic"
tokenizer = AutoTokenizer.from_pretrained(bert_name)
MBERT = AutoModel.from_pretrained(bert_name)

##Load/Store datasets for using the same validation set for all models
###Creating and storing datasets

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

# Taking 2000 samples for validation
val_idx = np.random.choice(len(sentences), size=2000, replace=False)
df_val = pd.DataFrame({"TEXT": sentences[val_idx], "LABEL": labels[val_idx]})
train_idx = list(set(range(len(sentences))) - set(val_idx))
df_train = pd.DataFrame({"TEXT": sentences[train_idx], "LABEL": labels[train_idx]})

# Translate validation set
translator = Translator()
df_val["TRANSLATE"] = df_val["TEXT"].apply(lambda t: translator.translate(t, src="ar", dest="en").text)

In [ ]:
import pickle

train_path = "/content/drive/My Drive/Colab Notebooks/NLP/Project/arabic_train_dataset.pkl"
val_path = "/content/drive/My Drive/Colab Notebooks/NLP/Project/arabic_validation_dataset.pkl"

with open(train_path, "wb") as f:
  pickle.dump(df_train, f)

with open(val_path, "wb") as f:
  pickle.dump(df_val, f)

In [ ]:
df_train.head()

,TEXT,LABEL
0,عجز الموازنه وصل ل93.7 % من الناتج المحلي يعني...,anger
1,كتنا نيله ف حظنا الهباب xD,sadness
2,جميعنا نريد تحقيق اهدافنا لكن تونس تالقت في حر...,joy
3,الاوليمبياد نظامها مختلف .. ومواعيد المونديال ...,none
4,دلوقتي عندها اربع سنين .... ومحدش يقدر يكلمها ...,surprise


###Loading datasets

In [ ]:
import pickle

train_path = "/content/drive/My Drive/Colab Notebooks/NLP/Project/arabic_train_dataset.pkl"
val_path = "/content/drive/My Drive/Colab Notebooks/NLP/Project/arabic_validation_dataset.pkl"

with open(train_path, "rb") as f:
  df_train = pickle.load(f)

with open(val_path, "rb") as f:
  df_val = pickle.load(f)

print(df_val["LABEL"].value_counts())

none        314
anger       286
sadness     264
joy         258
fear        250
love        220
sympathy    206
surprise    202
Name: LABEL, dtype: int64


##Preparing data and model
###Training the model afterwards

In [ ]:
input_ids_train, attention_masks_train, labels_train = tokenize(tokenizer, df_train["TEXT"].values, df_train["LABEL"].values, is_emotions=True)
input_ids_val, attention_masks_val, labels_val = tokenize(tokenizer, df_val["TEXT"].values, df_val["LABEL"].values, is_emotions=True)

Original:  عجز الموازنه وصل ل93.7 % من الناتج المحلي يعني لسه اقل من 7 % و نفلس و بهايم لسه يتابعوا الاوليمبياد
Token IDs: tensor([    2, 10126,  2277,  3573,  1023,  3685,   247, 23590,    18,    27,
            9,  1726, 13800,  5419,  3503,  3064,  1023,  4044,  1726,    27,
            9,   251,  3042, 11746,   251,  2281,  1782,  3064,  1023, 15229,
         1751, 11318, 21017,     3,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0])
Original:  فين جدول فعاليات الاوليمبياد ؟
Token IDs: tensor([    2,  6357,  8797,  5964, 11318, 21017,   221,     3,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
       

In [ ]:
train_dataloader, _ = load_split_dataset(input_ids_train, attention_masks_train, labels_train, to_split=False)
validation_dataloader, _ = load_split_dataset(input_ids_val, attention_masks_val, labels_val, to_split=False)

In [ ]:
import torch
import torch.nn as nn
from transformers import BertModel

class ArabicEmotionClassifier(nn.Module):

    def __init__(self, bert_model, freeze_bert = True):
        super(ArabicEmotionClassifier, self).__init__()
        #Instantiating BERT model object 
        self.bert_layer = bert_model
        
        #Freeze bert layers
        if freeze_bert:
            for p in self.bert_layer.parameters():
                p.requires_grad = False
        
        #Classification layer
        self.cls_layer = nn.Linear(768, NUM_LABELS)

    def forward(self, seq, attn_masks):
        '''
        Inputs:
            -seq : Tensor of shape [B, T] containing token ids of sequences
            -attn_masks : Tensor of shape [B, T] containing attention masks to be used to avoid contibution of PAD tokens
        '''

        #Feeding the input to BERT model to obtain contextualized representations
        _, hidden = self.bert_layer(seq, attention_mask = attn_masks)

        outputs = self.cls_layer(hidden)

        return outputs

In [ ]:
net = ArabicEmotionClassifier(MBERT, freeze_bert = False).cuda()

In [ ]:
import torch.nn as nn
import torch.optim as optim

criterion = nn.MSELoss()
opti = optim.Adam(net.parameters(), lr = 2e-5)

In [ ]:
train(net, criterion, opti, train_dataloader, validation_dataloader)

Iteration 1 of epoch 1 complete. Loss : 0.23892122507095337
Iteration 2 of epoch 1 complete. Loss : 0.15164636075496674
Iteration 3 of epoch 1 complete. Loss : 0.13975930213928223
Iteration 4 of epoch 1 complete. Loss : 0.12712720036506653
Iteration 5 of epoch 1 complete. Loss : 0.12382304668426514
Iteration 6 of epoch 1 complete. Loss : 0.10660453885793686
Iteration 7 of epoch 1 complete. Loss : 0.11512231826782227
Iteration 8 of epoch 1 complete. Loss : 0.11069299280643463
Iteration 9 of epoch 1 complete. Loss : 0.11420755833387375
Iteration 10 of epoch 1 complete. Loss : 0.11047864705324173
Iteration 11 of epoch 1 complete. Loss : 0.09876232594251633
Iteration 12 of epoch 1 complete. Loss : 0.1129668727517128
Iteration 13 of epoch 1 complete. Loss : 0.10926101356744766
Iteration 14 of epoch 1 complete. Loss : 0.08509314060211182
Iteration 15 of epoch 1 complete. Loss : 0.10287350416183472
Iteration 16 of epoch 1 complete. Loss : 0.10062941163778305
Iteration 17 of epoch 1 complete. 

##Results

In [ ]:
to_concat = []
with torch.no_grad():
    for data in validation_dataloader:
        sentences, attention_masks, labels = data
        outputs = net(sentences.cuda(), attention_masks.cuda())
        df = pd.DataFrame({"Sentence": [tokenizer.decode(s, skip_special_tokens = True) for s in sentences], "Real": [LABELS[l.argmax()] for l in labels], "Predicted": [LABELS[o.argmax()] for o in outputs]})
        to_concat.append(df)

arabic_validation_df = pd.concat(to_concat, ignore_index=True)

In [ ]:
pd.set_option("display.max_colwidth", 200)
display(arabic_validation_df)

,Sentence,Real,Predicted
0,الستات رافعين اسم مصر بره يا خيبتنا التقيله يا رجاله مصر الف مبروك لمصر فوز ساره سمير ببرونزيه رفع الاثقال في الاوليمبياد 👍,joy,joy
1,وبعد انتهاء المباراه احييك علی نظرتك الثاقبه وخفه دم بمكانها فعلا من سجل هو ناصر القحطاني????????????,joy,joy
2,هو في عاقل في مصر متفايل ؟!!!,surprise,surprise
3,علي مدار الساعه الله ينصركم ويثبت اقدامكم اجعلو التكبير سلاحكم فهي تزيد معنوياتكم وترعب عدوكم الله اكبر الله اكبر,sympathy,sympathy
4,بس خايفه اقولها والله وقلبي موش مطوعني عشان دا موش الاهلي بتاعنا بس الحمد لله ع ال3نقط ع الحلوه والمره معاه ❤ alahly,fear,fear
...,...,...,...
1995,سواء محمد او كريم واضح انهم مكسورين لكن محمد قوي وقادر يتماسك علي الاقل قدام الناس ، ربنا يصبرهم ويغفر للساحر محمود …,sympathy,sympathy
1996,يسعد صباحك ايش فيك شكلك خايفه,fear,fear
1997,rm : لماظه بحبك انا : لا امل ولا يتخللني ريب و كنت لي الهدايه بعد تلك الغوايه..! غــرام,love,love
1998,العاب القوي مكناتها في الاوليمبياد اعلي من السباحه برده,none,none


In [ ]:
from sklearn.metrics import precision_recall_fscore_support
from collections import defaultdict

def calcualte_measures(real, pred):
  """
  Calculating scores.

  Parameters
  ----------
  real : pd.Series
    Real labels.
  pred : pd.Series
    Predicted labels.

  Returns
  -------
  defaultdict
    Results, emotion to scores.
  """
  res = defaultdict(dict)
  measures = precision_recall_fscore_support(real, pred, labels=LABELS, average=None)
  for i, label in enumerate(LABELS):
    res[label]["precision"] = measures[0][i]
    res[label]["recall"] = measures[1][i]
    res[label]["f-score"] = measures[2][i]
  
  total_measure = precision_recall_fscore_support(real, pred, average="micro")
  res["total"] = {"precision": total_measure[0], "recall": total_measure[1], "f-score": total_measure[2]}

  return res

In [ ]:
calcualte_measures(arabic_validation_df["Real"], arabic_validation_df["Predicted"])

defaultdict(dict,
            {'anger': {'f-score': 0.686456400742115,
              'precision': 0.7312252964426877,
              'recall': 0.6468531468531469},
             'fear': {'f-score': 0.9209486166007905,
              'precision': 0.91015625,
              'recall': 0.932},
             'joy': {'f-score': 0.5739130434782609,
              'precision': 0.6534653465346535,
              'recall': 0.5116279069767442},
             'love': {'f-score': 0.7682119205298011,
              'precision': 0.7467811158798283,
              'recall': 0.7909090909090909},
             'none': {'f-score': 0.7524475524475525,
              'precision': 0.6708229426433915,
              'recall': 0.856687898089172},
             'sadness': {'f-score': 0.5,
              'precision': 0.5673076923076923,
              'recall': 0.44696969696969696},
             'surprise': {'f-score': 0.6008771929824561,
              'precision': 0.5393700787401575,
              'recall': 0.6782178217821783

In [ ]:
from collections import Counter

# Compositions
print("Real: ", Counter(arabic_validation_df["Real"]))
print("Predicted: ", Counter(arabic_validation_df["Predicted"]))

Real:  Counter({'none': 314, 'anger': 286, 'sadness': 264, 'joy': 258, 'fear': 250, 'love': 220, 'sympathy': 206, 'surprise': 202})
Predicted:  Counter({'none': 401, 'fear': 256, 'surprise': 254, 'anger': 253, 'love': 233, 'sadness': 208, 'joy': 202, 'sympathy': 193})


In [ ]:
len(validation_df[arabic_validation_df["Real"] == arabic_validation_df["Predicted"]])

1424

In [ ]:
len(arabic_validation_df)

2000

#EmoBank - Multilingual model
##Preparing the data and model, then training it

In [ ]:
sentences, labels = load_eb_arabic_dataset()

In [ ]:
bert_name="bert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(bert_name)
MBERT = AutoModel.from_pretrained(bert_name)

In [ ]:
input_ids, attention_masks, labels = tokenize(tokenizer, sentences, labels, is_emotions=False)

Original:  أتذكر ما قالت في رسالتي الأخيرة؟ "
Token IDs: tensor([  101,   759, 10502, 42172, 12441, 28598, 10502, 10210,   773, 49506,
        33691, 10461, 68631,   756,   107,   102,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0])


In [ ]:
train_dataloader, validation_dataloader = load_split_dataset(input_ids, attention_masks, labels, to_split=True)

In [ ]:
import torch
import torch.nn as nn
from transformers import BertModel

class VADClassifier(nn.Module):

    def __init__(self, bert_model, freeze_bert = True):
        super(VADClassifier, self).__init__()
        #Instantiating BERT model object 
        self.bert_layer = bert_model
        
        #Freeze bert layers
        if freeze_bert:
            for p in self.bert_layer.parameters():
                p.requires_grad = False
        
        #Classification layer
        self.cls_layer = nn.Linear(768, 3)

    def forward(self, seq, attn_masks):
        '''
        Inputs:
            -seq : Tensor of shape [B, T] containing token ids of sequences
            -attn_masks : Tensor of shape [B, T] containing attention masks to be used to avoid contibution of PAD tokens
        '''

        #Feeding the input to BERT model to obtain contextualized representations
        _, hidden = self.bert_layer(seq, attention_mask = attn_masks)

        outputs = self.cls_layer(hidden)

        return outputs

In [ ]:
net = VADClassifier(MBERT, freeze_bert = False).cuda()

In [ ]:
import torch.nn as nn
import torch.optim as optim

criterion = nn.MSELoss()
opti = optim.Adam(net.parameters(), lr = 2e-5)

In [ ]:
train(net, criterion, opti, train_dataloader, validation_dataloader, is_emotions=False)

Iteration 1 of epoch 1 complete. Loss : 0.09268004860037775
Iteration 2 of epoch 1 complete. Loss : 0.04714894552157836
Iteration 3 of epoch 1 complete. Loss : 0.047907450512409515
Iteration 4 of epoch 1 complete. Loss : 0.025533813452804466
Iteration 5 of epoch 1 complete. Loss : 0.024372105629844414
Iteration 6 of epoch 1 complete. Loss : 0.030091505715451106
Iteration 7 of epoch 1 complete. Loss : 0.036439626095800474
Iteration 8 of epoch 1 complete. Loss : 0.038760290396767816
Iteration 9 of epoch 1 complete. Loss : 0.034197919843069274
Iteration 10 of epoch 1 complete. Loss : 0.030987685885989255
Iteration 11 of epoch 1 complete. Loss : 0.029170399578461195
Iteration 12 of epoch 1 complete. Loss : 0.03189885307452512
Iteration 13 of epoch 1 complete. Loss : 0.052166019383854574
Iteration 14 of epoch 1 complete. Loss : 0.04775937126188418
Iteration 15 of epoch 1 complete. Loss : 0.03525838825496185
Iteration 16 of epoch 1 complete. Loss : 0.035285764764869276
Iteration 17 of epoch 

##Results
###First try: evaluating the model with EmoBank validation set

In [ ]:
to_concat = []
with torch.no_grad():
    for data in validation_dataloader:
        sentences, attention_masks, labels = data
        outputs = net(sentences.cuda(), attention_masks.cuda())
        df = pd.DataFrame({"Sentence": [tokenizer.decode(s, skip_special_tokens = True) for s in sentences], "Real VAD": [l.data.tolist() for l in labels], "Predicted VAD": [o.data.tolist() for o in outputs]})
        to_concat.append(df)

emo_arabic_validation_df1 = pd.concat(to_concat, ignore_index=True)

In [ ]:
pd.set_option("display.max_colwidth", 200)
display(emo_arabic_validation_df1)

,Sentence,Real VAD,Predicted VAD
0,مجموعة الخماسي هنا هي حصريًا فئة من طبقات الملعب ، وتجسيداتها طوال الحركة خالية من أي دلالات مرتبطة بمقياس الخماسي.,"[0.05882352941176493, -0.15384615384615374, 0.09090909090909127]","[0.10539770126342773, -0.1852266490459442, 0.10901886224746704]"
1,ومع ذلك ، قالت توني لاما إنها ستقدم العرض على الفور إلى لجنة خاصة من مجلس إدارة الشركة.,"[0.05882352941176493, -0.16923076923076952, -0.09090909090909084]","[0.29941320419311523, -0.07646647095680237, 0.2912669777870178]"
2,كلمات من الحكمة,"[0.20588235294117666, -0.0769230769230771, 0.3223140495867769]","[0.05661332234740257, -0.22219808399677277, -0.028917957097291946]"
3,يتم نسخ البرنامج من قبل مدن أخرى وجذب الطلاب من جميع مناحي الحياة ، الذين تتراوح أعمارهم بين 16 و 68.,"[0.4117647058823533, 0.15384615384615374, 0.25619834710743783]","[0.09856824576854706, -0.17731939256191254, 0.11881101876497269]"
4,الولايات المتحدة تفتح الباب للاجئين العراقيين,"[0.11764705882352965, 0.0, 0.17355371900826455]","[-0.148769810795784, 0.14778640866279602, 0.14797230064868927]"
...,...,...,...
399,أخبر عائلاتنا وأطفالنا أنك تريدهم أن يحصلوا على نفس أنواع ذكريات الأعياد التي ستقضيها.,"[0.2941176470588238, -0.15384615384615374, 0.00826446280991755]","[0.04200374335050583, -0.02960583008825779, 0.13822568953037262]"
400,العثور على المزيد من بقايا البشر في جراوند زيرو,"[-0.33529411764705874, -0.0769230769230771, -0.17355371900826455]","[0.10910221934318542, 0.024009963497519493, 0.06412152945995331]"
401,روسيا تحذر الولايات المتحدة من قواعد الصواريخ في أوروبا الشرقية,"[0.05882352941176493, 0.0, 0.33884297520661155]","[-0.22433972358703613, -0.04169505834579468, 0.010601135902106762]"
402,"إنه مثل القول "" لم أركل الكلب ، بل سقطت أضلاعه على حذائي "".","[-0.08823529411764675, -0.16923076923076952, 0.3223140495867769]","[-0.27169549465179443, 0.03320666402578354, -0.036596670746803284]"


In [ ]:
emo_arabic_validation_df1["Real"] = emo_arabic_validation_df1["Real VAD"].apply(convertVadToEmotion)
emo_arabic_validation_df1["Predicted"] = emo_arabic_validation_df1["Predicted VAD"].apply(convertVadToEmotion)

In [ ]:
emo_arabic_validation_df1.head()

,Sentence,Real VAD,Predicted VAD,Real,Predicted,Real (without none),Predicted (without none)
0,مجموعة الخماسي هنا هي حصريًا فئة من طبقات الملعب ، وتجسيداتها طوال الحركة خالية من أي دلالات مرتبطة بمقياس الخماسي.,"[0.05882352941176493, -0.15384615384615374, 0.09090909090909127]","[0.10539770126342773, -0.1852266490459442, 0.10901886224746704]",none,none,sympathy,sympathy
1,ومع ذلك ، قالت توني لاما إنها ستقدم العرض على الفور إلى لجنة خاصة من مجلس إدارة الشركة.,"[0.05882352941176493, -0.16923076923076952, -0.09090909090909084]","[0.29941320419311523, -0.07646647095680237, 0.2912669777870178]",none,none,sadness,sympathy
2,كلمات من الحكمة,"[0.20588235294117666, -0.0769230769230771, 0.3223140495867769]","[0.05661332234740257, -0.22219808399677277, -0.028917957097291946]",none,none,sympathy,sadness
3,يتم نسخ البرنامج من قبل مدن أخرى وجذب الطلاب من جميع مناحي الحياة ، الذين تتراوح أعمارهم بين 16 و 68.,"[0.4117647058823533, 0.15384615384615374, 0.25619834710743783]","[0.09856824576854706, -0.17731939256191254, 0.11881101876497269]",sympathy,none,sympathy,sympathy
4,الولايات المتحدة تفتح الباب للاجئين العراقيين,"[0.11764705882352965, 0.0, 0.17355371900826455]","[-0.148769810795784, 0.14778640866279602, 0.14797230064868927]",none,none,sympathy,anger


In [ ]:
calcualte_measures(emo_arabic_validation_df1["Real"], emo_arabic_validation_df1["Predicted"])

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


defaultdict(dict,
            {'anger': {'f-score': 0.18181818181818182,
              'precision': 0.25,
              'recall': 0.14285714285714285},
             'fear': {'f-score': 0.0, 'precision': 0.0, 'recall': 0.0},
             'joy': {'f-score': 0.0, 'precision': 0.0, 'recall': 0.0},
             'love': {'f-score': 0.0, 'precision': 0.0, 'recall': 0.0},
             'none': {'f-score': 0.9384404924760602,
              'precision': 0.9026315789473685,
              'recall': 0.9772079772079773},
             'sadness': {'f-score': 0.3333333333333333,
              'precision': 0.5,
              'recall': 0.25},
             'surprise': {'f-score': 0.4,
              'precision': 0.5,
              'recall': 0.3333333333333333},
             'sympathy': {'f-score': 0.23529411764705885,
              'precision': 0.3333333333333333,
              'recall': 0.18181818181818182},
             'total': {'f-score': 0.8712871287128713,
              'precision': 0.8712871287128713

In [ ]:
len(emo_arabic_validation_df1[emo_arabic_validation_df1["Real"] == emo_arabic_validation_df1["Predicted"]])

352

In [ ]:
len(emo_arabic_validation_df1)

404

In [ ]:
from collections import Counter

# Composition
print("Real: ", Counter(emo_arabic_validation_df1["Real"]))
print("Predicted: ", Counter(emo_arabic_validation_df1["Predicted"]))

Real:  Counter({'none': 351, 'sympathy': 22, 'sadness': 12, 'anger': 7, 'fear': 6, 'joy': 3, 'surprise': 3})
Predicted:  Counter({'none': 380, 'sympathy': 12, 'sadness': 6, 'anger': 4, 'surprise': 2})


In [ ]:
# Without none emotion
emo2vad = {
    "anger": (-0.51, 0.59, 0.25), #82
    "fear": (-0.64, 0.60, -0.43), #101
    "joy": (0.76, 0.48, 0.35), #20
    "love": (0.87, 0.54, -0.18), #47
    #"none": (0, 0, 0),
    "sadness": (-0.63, -0.27, -0.33), #151
    "surprise": (0.40, 0.67, -0.13), #52
    "sympathy": (0.64, 0.35, 0.24), #34
}
emo_arabic_validation_df1["Real (without none)"] = emo_arabic_validation_df1["Real VAD"].apply(convertVadToEmotion)
emo_arabic_validation_df1["Predicted (without none)"] = emo_arabic_validation_df1["Predicted VAD"].apply(convertVadToEmotion)
calcualte_measures(emo_arabic_validation_df1["Real (without none)"], emo_arabic_validation_df1["Predicted (without none)"])

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


defaultdict(dict,
            {'anger': {'f-score': 0.36111111111111105,
              'precision': 0.3023255813953488,
              'recall': 0.4482758620689655},
             'fear': {'f-score': 0.0, 'precision': 0.0, 'recall': 0.0},
             'joy': {'f-score': 0.0, 'precision': 0.0, 'recall': 0.0},
             'love': {'f-score': 0.0, 'precision': 0.0, 'recall': 0.0},
             'none': {'f-score': 0.0, 'precision': 0.0, 'recall': 0.0},
             'sadness': {'f-score': 0.5266457680250783,
              'precision': 0.5060240963855421,
              'recall': 0.5490196078431373},
             'surprise': {'f-score': 0.08,
              'precision': 0.14285714285714285,
              'recall': 0.05555555555555555},
             'sympathy': {'f-score': 0.6141732283464568,
              'precision': 0.6223404255319149,
              'recall': 0.6062176165803109},
             'total': {'f-score': 0.5321782178217822,
              'precision': 0.5321782178217822,
             

In [ ]:
len(emo_arabic_validation_df1[emo_arabic_validation_df1["Real (without none)"] == emo_arabic_validation_df1["Predicted (without none)"]])

215

In [ ]:
from collections import Counter

print("Real: ", Counter(emo_arabic_validation_df1["Real (without none)"]))
print("Predicted: ", Counter(emo_arabic_validation_df1["Predicted (without none)"]))

Real:  Counter({'sympathy': 193, 'sadness': 153, 'anger': 29, 'surprise': 18, 'fear': 8, 'joy': 3})
Predicted:  Counter({'sympathy': 188, 'sadness': 166, 'anger': 43, 'surprise': 7})


###Second try: evaluating the model with the Arabic validation set

In [ ]:
# Loading the validation set
val_path = "/content/drive/My Drive/Colab Notebooks/NLP/Project/arabic_validation_dataset.pkl"
with open(val_path, "rb") as f:
  df = pickle.load(f)

print("Validation set size:", len(df))
print(df["LABEL"].value_counts())
input_ids, attention_masks, labels = tokenize(tokenizer, df["TEXT"].values, df["LABEL"], is_emotions=True)
validation_dataloader, _ = load_split_dataset(input_ids, attention_masks, labels)

Validation set size: 2000
none        314
anger       286
sadness     264
joy         258
fear        250
love        220
sympathy    206
surprise    202
Name: LABEL, dtype: int64
Original:  فين جدول فعاليات الاوليمبياد ؟
Token IDs: tensor([   101,  10210,  10582, 102687,  69420,  14964,  59901,  29452,  20556,
         56064,  10658,    756,    102,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0])


In [ ]:
to_concat = []
with torch.no_grad():
    for data in validation_dataloader:
        sentences, attention_masks, labels = data
        outputs = net(sentences.cuda(), attention_masks.cuda())
        df = pd.DataFrame({"Sentence": [tokenizer.decode(s, skip_special_tokens = True) for s in sentences], "Real": [LABELS[l.argmax()] for l in labels], "Predicted VAD": [o.data.tolist() for o in outputs]})
        to_concat.append(df)

emo_arabic_validation_df2 = pd.concat(to_concat, ignore_index=True)

In [ ]:
# With none emotion
emo2vad = {
    "anger": (-0.51, 0.59, 0.25), #82
    "fear": (-0.64, 0.60, -0.43), #101
    "joy": (0.76, 0.48, 0.35), #20
    "love": (0.87, 0.54, -0.18), #47
    "none": (0, 0, 0),
    "sadness": (-0.63, -0.27, -0.33), #151
    "surprise": (0.40, 0.67, -0.13), #52
    "sympathy": (0.64, 0.35, 0.24), #34
}
emo_arabic_validation_df2["Predicted"] = emo_arabic_validation_df2["Predicted VAD"].apply(convertVadToEmotion)
emo_arabic_validation_df2.head()

,Sentence,Real,Predicted VAD,Predicted
0,فرص الجزائر!!!,surprise,"[0.0802202820777893, 0.5449303388595581, 0.14642930030822754]",surprise
1,يارب هاهم اطفال ينادوك يالله فرج عنهم يارب,sympathy,"[-0.020780259743332863, 0.21091151237487793, 0.026089243590831757]",none
2,ياه اوحش حاجه انك تظن في حاجه وتنسي ان بعد الظن اثم,fear,"[-0.001957666128873825, 0.13369067013263702, -0.06448590755462646]",none
3,لاعبنا الجديد 3 1 ترددها 12226,none,"[0.08828245103359222, -0.05727037787437439, 0.09682203084230423]",none
4,مسائك اجمل,joy,"[0.1541379690170288, -0.11255783587694168, 0.06775268167257309]",none


In [ ]:
calcualte_measures(emo_arabic_validation_df2["Real"], emo_arabic_validation_df2["Predicted"])

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


defaultdict(dict,
            {'anger': {'f-score': 0.006472491909385114,
              'precision': 0.043478260869565216,
              'recall': 0.0034965034965034965},
             'fear': {'f-score': 0.031128404669260697,
              'precision': 0.5714285714285714,
              'recall': 0.016},
             'joy': {'f-score': 0.0, 'precision': 0.0, 'recall': 0.0},
             'love': {'f-score': 0.0, 'precision': 0.0, 'recall': 0.0},
             'none': {'f-score': 0.29425723777883245,
              'precision': 0.17289459007250418,
              'recall': 0.9872611464968153},
             'sadness': {'f-score': 0.0, 'precision': 0.0, 'recall': 0.0},
             'surprise': {'f-score': 0.13953488372093023,
              'precision': 0.32142857142857145,
              'recall': 0.0891089108910891},
             'sympathy': {'f-score': 0.13580246913580246,
              'precision': 0.1864406779661017,
              'recall': 0.10679611650485436},
             'total': {'f-sc

In [ ]:
len(emo_arabic_validation_df2[emo_arabic_validation_df2["Real"] == emo_arabic_validation_df2["Predicted"]])

355

In [ ]:
len(emo_arabic_validation_df2)

2000

In [ ]:
from collections import Counter

print("Real: ", Counter(emo_arabic_validation_df2["Real"]))
print("Predicted: ", Counter(emo_arabic_validation_df2["Predicted"]))

Real:  Counter({'none': 314, 'anger': 286, 'sadness': 264, 'joy': 258, 'fear': 250, 'love': 220, 'sympathy': 206, 'surprise': 202})
Predicted:  Counter({'none': 1793, 'sympathy': 118, 'surprise': 56, 'anger': 23, 'fear': 7, 'sadness': 2, 'joy': 1})


In [ ]:
# Without none emotion
emo2vad = {
    "anger": (-0.51, 0.59, 0.25), #82
    "fear": (-0.64, 0.60, -0.43), #101
    "joy": (0.76, 0.48, 0.35), #20
    "love": (0.87, 0.54, -0.18), #47
    #"none": (0, 0, 0),
    "sadness": (-0.63, -0.27, -0.33), #151
    "surprise": (0.40, 0.67, -0.13), #52
    "sympathy": (0.64, 0.35, 0.24), #34
}
df = emo_arabic_validation_df2[emo_arabic_validation_df2["Real"] != "none"]
df["Predicted (without none)"] = df["Predicted VAD"].apply(convertVadToEmotion)
calcualte_measures(df["Real"], df["Predicted (without none)"])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


defaultdict(dict,
            {'anger': {'f-score': 0.20032310177705975,
              'precision': 0.18618618618618618,
              'recall': 0.21678321678321677},
             'fear': {'f-score': 0.031007751937984496,
              'precision': 0.5,
              'recall': 0.016},
             'joy': {'f-score': 0.0, 'precision': 0.0, 'recall': 0.0},
             'love': {'f-score': 0.0, 'precision': 0.0, 'recall': 0.0},
             'none': {'f-score': 0.0, 'precision': 0.0, 'recall': 0.0},
             'sadness': {'f-score': 0.19614711033274956,
              'precision': 0.18241042345276873,
              'recall': 0.21212121212121213},
             'surprise': {'f-score': 0.18738049713193114,
              'precision': 0.1526479750778816,
              'recall': 0.24257425742574257},
             'sympathy': {'f-score': 0.20173535791757047,
              'precision': 0.12988826815642457,
              'recall': 0.45145631067961167},
             'total': {'f-score': 0.156583629

In [ ]:
len(df[df["Real"] == df["Predicted (without none)"]])

264

In [ ]:
len(df)

1686

In [ ]:
from collections import Counter

print("Real: ", Counter(df["Real"]))
print("Predicted: ", Counter(df["Predicted (without none)"]))

Real:  Counter({'anger': 286, 'sadness': 264, 'joy': 258, 'fear': 250, 'love': 220, 'sympathy': 206, 'surprise': 202})
Predicted:  Counter({'sympathy': 716, 'anger': 333, 'surprise': 321, 'sadness': 307, 'fear': 8, 'joy': 1})


#EmoBank - English
##Preparing data and model with English EmoBank, then training the model

In [ ]:
emo2vad = {
    "anger": (-0.51, 0.59, 0.25), #82
    "fear": (-0.64, 0.60, -0.43), #101
    "joy": (0.76, 0.48, 0.35), #20
    "love": (0.87, 0.54, -0.18), #47
    "none": (0, 0, 0),
    "sadness": (-0.63, -0.27, -0.33), #151
    "surprise": (0.40, 0.67, -0.13), #52
    "sympathy": (0.64, 0.35, 0.24), #34
}

In [ ]:
sentences, labels = load_eb_english_emotion_dataset()

In [ ]:
bert_name="bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(bert_name)
MBERT = AutoModel.from_pretrained(bert_name)

In [ ]:
input_ids, attention_masks, labels = tokenize(tokenizer, sentences, labels, is_emotions=True)

Original:  Remember what she said in my last letter? "
Token IDs: tensor([ 101, 9498, 1184, 1131, 1163, 1107, 1139, 1314, 2998,  136,  107,  102,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0])


In [ ]:
train_dataloader, _ = load_split_dataset(input_ids, attention_masks, labels)

# Loading validation set from the Arabic dataset
val_path = "/content/drive/My Drive/Colab Notebooks/NLP/Project/arabic_validation_dataset.pkl"
with open(val_path, "rb") as f:
  df_val = pickle.load(f)


import unicodedata
from string import punctuation

def is_all_in_english(s):
  """
  Whether the string is completely in English.

  Parameters
  ----------
  s : str
    String.

  Returns
  -------
  bool
    Whether the input string is completely in English.
  """
  for c in s:
    if c in punctuation or c == " ":
      continue
    try:
      name = unicodedata.name(c).lower()
    except:
      return False
    if 'arabic' in name:
      return False
  return True

# Keeping only samples that are completely in English
df = df_val[df_val["TRANSLATE"].apply(is_all_in_english)]
print("Validation set size:", len(df))
print(df["LABEL"].value_counts())
input_ids, attention_masks, labels = tokenize(tokenizer, df["TRANSLATE"].values, df["LABEL"], is_emotions=True)
validation_dataloader, _ = load_split_dataset(input_ids, attention_masks, labels)

Validation set size: 1172
anger       184
none        179
sadness     158
fear        148
joy         132
love        129
sympathy    124
surprise    118
Name: LABEL, dtype: int64
Original:  Where is the Olympiad schedule?
Token IDs: tensor([  101,  2777,  1110,  1103, 18279,  1181,  6030,   136,   102,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0])


In [ ]:
net = ArabicEmotionClassifier(MBERT, freeze_bert = False).cuda()

In [ ]:
import torch.nn as nn
import torch.optim as optim

criterion = nn.MSELoss()
opti = optim.Adam(net.parameters(), lr = 2e-5)

In [ ]:
train(net, criterion, opti, train_dataloader, validation_dataloader, is_emotions=False)

Iteration 1 of epoch 1 complete. Loss : 0.3934088765999738
Iteration 2 of epoch 1 complete. Loss : 0.3198203058669239
Iteration 3 of epoch 1 complete. Loss : 0.29666965377326293
Iteration 4 of epoch 1 complete. Loss : 0.2314520391742208
Iteration 5 of epoch 1 complete. Loss : 0.16665012607880747
Iteration 6 of epoch 1 complete. Loss : 0.14791492731211575
Iteration 7 of epoch 1 complete. Loss : 0.13131816208120137
Iteration 8 of epoch 1 complete. Loss : 0.09979218592583022
Iteration 9 of epoch 1 complete. Loss : 0.0966240462528713
Iteration 10 of epoch 1 complete. Loss : 0.1195559085041431
Iteration 11 of epoch 1 complete. Loss : 0.05635337597207408
Iteration 12 of epoch 1 complete. Loss : 0.04105424025960702
Iteration 13 of epoch 1 complete. Loss : 0.053337772783020376
Iteration 14 of epoch 1 complete. Loss : 0.03721627685039247
Iteration 15 of epoch 1 complete. Loss : 0.019891445998970726
Iteration 16 of epoch 1 complete. Loss : 0.03017097088156225
Iteration 17 of epoch 1 complete. Lo

##Results

In [ ]:
to_concat = []
with torch.no_grad():
    for data in validation_dataloader:
        sentences, attention_masks, labels = data
        outputs = net(sentences.cuda(), attention_masks.cuda())
        df = pd.DataFrame({"Sentence": [tokenizer.decode(s, skip_special_tokens = True) for s in sentences], "Real": [LABELS[l.argmax()] for l in labels], "Predicted": [LABELS[o.argmax()] for o in outputs]})
        to_concat.append(df)

emo_english_validation_df = pd.concat(to_concat, ignore_index=True)

In [ ]:
pd.set_option("display.max_colwidth", 200)
display(emo_english_validation_df)

,Sentence,Real,Predicted
0,"Your prayers to Thomas for a speedy recovery O God, Lord of peace, guide the people, bring down your wellness to every patient who complains of fatigue.",sympathy,none
1,"O Lord, whoever has a need in your security call, and you have pleasure in it, and happiness does not deprive him, O friend of mercy and benevolence",sympathy,sympathy
2,"If he invaded Kuwait, we have mercy on him, why do we respect him, Why!?",anger,anger
3,"My brother Abu Mutaib, what is the opposition to victory, al - Qaeda, ISIS, or the Turkish occupation army, or the destruction of the Levant, I don ’ t know?",anger,none
4,"Yousef9 : Why is there a division in the street after the Brotherhood, except for the animal who was released?!!!",surprise,anger
...,...,...,...
1167,"Justice is absent, truth is absent, and humanity is absent at the time of the strong ’ s rule over the weak. Is the Security Council composed of the rulers of the world, or is it made up of the wo...",anger,none
1168,"I am waiting for an important call, so I was old today, and I am afraid I heard the mobile : ( (",fear,none
1169,"Weird, in this time, women have become women, and some people have become women????????",surprise,none
1170,A message to everyone or alone with his dirt insulted Basem Yusef or accused them of betraying him and his workers and that he didn ’ t not understand Basem Yusef.,anger,none


In [ ]:
calcualte_measures(emo_english_validation_df["Real"], emo_english_validation_df["Predicted"])

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


defaultdict(dict,
            {'anger': {'f-score': 0.09049773755656108,
              'precision': 0.2702702702702703,
              'recall': 0.05434782608695652},
             'fear': {'f-score': 0.21965317919075145,
              'precision': 0.76,
              'recall': 0.12837837837837837},
             'joy': {'f-score': 0.0, 'precision': 0.0, 'recall': 0.0},
             'love': {'f-score': 0.0, 'precision': 0.0, 'recall': 0.0},
             'none': {'f-score': 0.30141843971631205,
              'precision': 0.1791359325605901,
              'recall': 0.9497206703910615},
             'sadness': {'f-score': 0.048192771084337345,
              'precision': 0.5,
              'recall': 0.02531645569620253},
             'surprise': {'f-score': 0.208955223880597,
              'precision': 0.875,
              'recall': 0.11864406779661017},
             'sympathy': {'f-score': 0.06896551724137931,
              'precision': 0.06569343065693431,
              'recall': 0.07258064

In [ ]:
from collections import Counter

# Composition
print("Real: ", Counter(emo_english_validation_df["Real"]))
print("Predicted: ", Counter(emo_english_validation_df["Predicted"]))

Real:  Counter({'anger': 184, 'none': 179, 'sadness': 158, 'fear': 148, 'joy': 132, 'love': 129, 'sympathy': 124, 'surprise': 118})
Predicted:  Counter({'none': 949, 'sympathy': 137, 'anger': 37, 'fear': 25, 'surprise': 16, 'sadness': 8})


In [ ]:
len(emo_english_validation_df[emo_english_validation_df["Real"] == emo_english_validation_df["Predicted"]])

226

In [ ]:
len(emo_english_validation_df)

1172